# 目的
- チョコボールを認識する
- ChainerCV利用

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np

import chainer

from chainercv.chainer_experimental.datasets.sliceable import TupleDataset
from chainercv.links import FasterRCNNVGG16
from chainercv.links.model.faster_rcnn import FasterRCNNTrainChain
from chainer.datasets import TransformDataset
from chainercv import transforms
from chainer import training
from chainer.training import extensions

/home/tokita/.pyenv/versions/anaconda2-4.4.0/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
HOME = '/home/tokita/workspace/etc/ToolSamples/chainer-CV'

# BoundingBoxDataSet

In [3]:
def getClasses(classes_file):
    # カテゴリファイル
    classes = list()
    with open(classes_file) as fd:
        for one_line in fd.readlines():
            cl = one_line.split('\n')[0]
            classes.append(cl)
    return classes

In [4]:
data_dir = os.path.join(HOME, 'data/chocoball')

file_img_set = os.path.join(data_dir, 'images.npy')
file_bbox_set = os.path.join(data_dir, 'bounding_box_data.npy')
file_object_ids = os.path.join(data_dir, 'object_ids.npy')

file_classes = os.path.join(data_dir, 'classes.txt')

In [5]:
imgs = np.load(file_img_set)
bboxs = np.load(file_bbox_set)
objectIDs = np.load(file_object_ids)
classes = getClasses(file_classes)

In [7]:
print(imgs.shape)
print(classes)

(4, 3, 302, 403)
['choco-ball', 'choco-package']


In [8]:
dataset = TupleDataset(('img', imgs), ('bbox', bboxs), ('label', objectIDs))
print(dataset.keys)
print(len(dataset))

('img', 'bbox', 'label')
4


# load Faster R-CNN Model

In [10]:
faster_rcnn = FasterRCNNVGG16(n_fg_class=len(classes), 
                              pretrained_model='imagenet')

In [11]:
# プリセットされた閾値を使用する
faster_rcnn.use_preset('evaluate')

# Set Chain

In [12]:
gpu_id = 4

In [13]:
model = FasterRCNNTrainChain(faster_rcnn)
chainer.cuda.get_device_from_id(gpu_id).use()
model.to_gpu()

# Optimizer

In [14]:
optimizer = chainer.optimizers.MomentumSGD(lr=0.001, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(rate=0.0005))

# set data

In [15]:
class Transform(object):

    def __init__(self, faster_rcnn):
        self.faster_rcnn = faster_rcnn

    def __call__(self, in_data):
        img, bbox, label = in_data
        _, H, W = img.shape
        img = self.faster_rcnn.prepare(img)
        _, o_H, o_W = img.shape
        scale = o_H / H
        bbox = transforms.resize_bbox(bbox, (H, W), (o_H, o_W))

        # horizontally flip
        img, params = transforms.random_flip(
            img, x_random=True, return_param=True)
        bbox = transforms.flip_bbox(
            bbox, (o_H, o_W), x_flip=params['x_flip'])

        return img, bbox, label, scale

In [16]:
train_data = TransformDataset(dataset, Transform(faster_rcnn))

In [17]:
train_iter = chainer.iterators.SerialIterator(
    train_data, batch_size=1)
test_iter = chainer.iterators.SerialIterator(
    dataset, batch_size=1, repeat=False, shuffle=False)

updater = chainer.training.updaters.StandardUpdater(
    train_iter, optimizer, device=gpu_id)

In [18]:
n_iter = 100
out_dir = './out'
trainer = training.Trainer(
    updater, (n_iter, 'iteration'), out=out_dir)

In [19]:
step_size = 100

trainer.extend(
    extensions.snapshot_object(model.faster_rcnn, 'snapshot_model.npz'),
    trigger=(n_iter, 'iteration'))
trainer.extend(extensions.ExponentialShift('lr', 0.1),
               trigger=(step_size, 'iteration'))

In [20]:
log_interval = 20, 'iteration'
plot_interval = 3000, 'iteration'
print_interval = 20, 'iteration'

In [21]:
trainer.extend(chainer.training.extensions.observe_lr(),
               trigger=log_interval)

trainer.extend(extensions.LogReport(trigger=log_interval))

trainer.extend(extensions.PrintReport(
    ['iteration', 'epoch', 'elapsed_time', 'lr',
     'main/loss',
     'main/roi_loc_loss',
     'main/roi_cls_loss',
     'main/rpn_loc_loss',
     'main/rpn_cls_loss',
     'validation/main/map',
    ]), trigger=print_interval)
trainer.extend(extensions.ProgressBar(update_interval=10))

trainer.extend(
    extensions.PlotReport(
        ['main/loss'],
        file_name='loss.png', trigger=plot_interval
    ),
    trigger=plot_interval
)

trainer.extend(extensions.dump_graph('main/loss'))

# Training

In [22]:
trainer.run()

     total [#####.............................................] 10.00%
this epoch [#########################.........................] 50.00%
        10 iter, 2 epoch / 100 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
iteration   epoch       elapsed_time  lr          main/loss   main/roi_loc_loss  main/roi_cls_loss  main/rpn_loc_loss  main/rpn_cls_loss  validation/main/map
20          5           6.82571       0.001       1.90057     0.497307           0.451982           0.386238           0.565043                                
     total [##########........................................] 20.00%
this epoch [..................................................]  0.00%
        20 iter, 5 epoch / 100 iterations
    3.5407 iters/sec. Estimated time to finish: 0:00:22.594702.
     total [###############...................................] 30.00%
this epoch [#########################.........................] 50.00%
        30 iter, 7 epoch / 100 iterations
    3.705